In [102]:
import requests
import time
import json
import pandas as pd


In [103]:
term = 'italian food'
location = 'Greenwich Village NY'
url_params = {  'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit' : 50
             }

In [104]:
# Your code here; use a function or loop to retrieve all the results from your original request

def rest_search(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    data = response.json()['businesses']
    return data



# df_rest = pd.DataFrame(rest_search(url_params, api_key))
# print(len(df_rest))
# df_rest

# print(len(rest_search(url_params, api_key)))
# rest_search(url_params, api_key)


In [105]:
def all_rests(url_params, api_key):
    url = 'https://api.yelp.com/v3/businesses/search'
    headers = {'Authorization': 'Bearer {}'.format(api_key)}
    response = requests.get(url, headers=headers, params=url_params)
    num = response.json()['total']
    print('{} total matches found.'.format(num))
    cur = 0
    results = []
    while cur < num and cur < 1000:
        url_params['offset'] = cur
        results.extend(rest_search(url_params, api_key))
        time.sleep(1) #Wait a second
        cur += 50
    return results


# df_all = pd.DataFrame(all_rests(url_params, api_key))
# print(len(df_all))
# df_all.head(10)


all_restaurants = all_rests(url_params, api_key)

1600 total matches found.


In [106]:
all_restaurants

[{'id': '16ZnHpuaaBt92XWeJHCC5A',
  'alias': 'olio-e-piú-new-york-3',
  'name': 'Olio e Piú',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/l4peUZMi7CLFcG3w3jnYvQ/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/olio-e-pi%C3%BA-new-york-3?adjust_creative=RUNeA23i4vSpUWOOA2zPTA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=RUNeA23i4vSpUWOOA2zPTA',
  'review_count': 1846,
  'categories': [{'alias': 'pizza', 'title': 'Pizza'},
   {'alias': 'italian', 'title': 'Italian'},
   {'alias': 'breakfast_brunch', 'title': 'Breakfast & Brunch'}],
  'rating': 4.0,
  'coordinates': {'latitude': 40.733826, 'longitude': -73.999852},
  'transactions': ['restaurant_reservation', 'pickup', 'delivery'],
  'price': '$$',
  'location': {'address1': '3 Greenwich Ave',
   'address2': None,
   'address3': '',
   'city': 'New York',
   'zip_code': '10014',
   'country': 'US',
   'state': 'NY',
   'display_address': ['3 Greenwich Ave', 'New York, NY 10014']},
  'ph

In [107]:

import config
import mysql.connector
config.host

'flatiron.cilucwxmxhor.us-east-2.rds.amazonaws.com'

In [55]:
cnx = mysql.connector .connect(
    host = config.host,
    user = config.username,
    passwd = config.password)

In [56]:
config.host
cursor = cnx.cursor()


In [57]:
import mysql.connector
from mysql.connector import errorcode

In [58]:
db_name = 'YELPAPI'

In [59]:
def create_database(cursor, database): #this is a function that tries to create a database
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)



    try:
        cursor.execute("USE {}".format(db_name)) #this will fail because we haven't created it yet
    except mysql.connector.Error as err:
        print("Database {} does not exists.".format(db_name))
        if err.errno == errorcode.ER_BAD_DB_ERROR: #if it fails, use this to make the DB
            create_database(cursor, db_name)
            print("Database {} created successfully.".format(db_name))
            cnx.database = db_name
        else:
            print(err)
            exit(1)

In [60]:
create_database(cursor, db_name)


In [61]:
DB_NAME = 'YELPAPI'
TABLES = {}
TABLES['Restaurants'] = (
    "CREATE TABLE Restaurants ("
    "  id INT(11) NOT NULL,"
    "  name varchar(32) NOT NULL,"
    "  price varchar(4) NOT NULL,"
    "  rating int(4) NOT NULL,"
    "  review_count INT(6) NOT NULL,"
    "  PRIMARY KEY (id)"
    ") ENGINE=InnoDB")

In [62]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.username,
    passwd = config.password,
    database = DB_NAME)

cursor = cnx.cursor()

In [63]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close() #VERY IMPORTANT TO CLOSE
cnx.close()

Creating table Restaurants: OK


In [64]:
cnx = mysql.connector .connect(
    host = config.host,
    user = config.username,
    passwd = config.password,
    database = db_name)
cursor = cnx.cursor()

In [65]:
cursor.close()
cnx.close()

In [66]:
for restaurant in all_restaurants:
    cursor.execute("""insert into Restaurants
    (id, name, price, rating, review_count) values (%s,%s,%s,%s,%s)""",
    (restaurant['id'], restaurant['name'], restaurant['price'], restaurant['rating'], restaurant['review_count']))
   
cursor.close()
cnx.close()

ProgrammingError: Cursor is not connected

In [108]:
id_list = []
for res_id in all_restaurants:
    id_list.append(res_id['id'])

In [109]:
id_list

['16ZnHpuaaBt92XWeJHCC5A',
 'V-FCEicobl5Su-R7EVJQNA',
 'yZJbealagfujEkgesAl2hA',
 'ETgJqJHV7BW6pIr9Ox74sA',
 'rp8S7scxOkVVeG7dNneMGQ',
 'WG639VkTjmK5dzydd1BBJA',
 'WHQpLvxkCK_BMtF8pIShnQ',
 'MbwsCB7echBlVwzfJ_9c9g',
 'hdiuRS9sVZSMReZm4oV5SA',
 '-hUvO0C0A-pGZdiSKvjoFw',
 'k5fd-RSdG4IuXnnGSQ5oOw',
 'XfOa_KyrV7JSFJMOKckGMg',
 'ujbH5MZQzC2SSuRF3XvNfQ',
 'qBGKigOGn_flCx1apxPzlA',
 '15k7iqFbhf4h8L01yggLqg',
 '7_hGsGEkHrjgAiYVjrCFnw',
 'bW3mY-voyav37nU0uaLmNQ',
 'OCTiJwvjoK81WoDwsTkFvA',
 'Pu09oBx1ekEZiFt90PsSIw',
 'IErfGS0DASSJfYU73m0Sqg',
 'MphfKkYFEe0BDQ6hZ0A5fA',
 'Ei5qjq0qmfNLkdVoO-S5Tw',
 'J0OeDwmh5JBhid0A5GvltA',
 'aHRcUI_UqFNV87GF7WRIDQ',
 'jdNdy-MwBtnXoBGC_JVvwQ',
 'I2lgw_7DUnwD92ND4PN-Ow',
 'XEugUtbw4rRmGr9S1XA-aQ',
 'ONEe1U1fNGT80jAEE0sCrg',
 '7oNuDMESKejMgJFlkQwxmQ',
 'h37t9rA06Sr4EetJjKrfzw',
 'qrcu1K6f8WGMODdq_A8unw',
 'BgGz5IdV53bGb4o7opR5zg',
 'hDj_0pbi4mFR5p1H1w6igw',
 'Vs8kQ4R381y27AH018gLpg',
 'aJJlY5xvqS180wtRIHEWqA',
 '04exhf1PZ79_Km9vHAzvfA',
 'jnHbdsqlTKlPcmJ8BCP9-g',
 

In [113]:
id_list[0]

'16ZnHpuaaBt92XWeJHCC5A'

In [125]:
def all_revs(api_key, id_list):
    results = []
    for id_rest in id_list:
        url = f'https://api.yelp.com/v3/businesses/{id_rest}/reviews'
        headers = {'Authorization': 'Bearer {}'.format(api_key)}
        response = requests.get(url, headers=headers)
        resreviews = response.json()['reviews']
        #iterate through the reviews to add the business_id to each
        #then iterate through the 'text' of reviews to take out the \n's
        for i in list(range(len(resreviews))):
            resreviews[i]['business_id'] = id_rest
            resreviews[i]['text'] = resreviews[i]['text'].replace('\n', '')            
            
        results.extend(resreviews)
        time.sleep(1)
    return results

In [123]:
api_key

'SHkhGqxiYEcd610uyBUylLMhMeJnZVidmjaZ_lFwO5EzWIvReF5ZT5XWAtZHEsINDORZBsN7ZY2Mo73hQoiAGx4A16OltG1j2NLH2vBuLjIEw2kyE0CkBP4VAfw5XXYx'

In [126]:
small_res = all_revs(api_key, id_list[0:2])

In [127]:
small_res

[{'id': 'B_NNbvsWvBeFXvoaKQh-rw',
  'url': 'https://www.yelp.com/biz/olio-e-pi%C3%BA-new-york-3?adjust_creative=RUNeA23i4vSpUWOOA2zPTA&hrid=B_NNbvsWvBeFXvoaKQh-rw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm_source=RUNeA23i4vSpUWOOA2zPTA',
  'text': 'I ordered various cheese and meats as an appetizer with my family - buratta was by far the best but pecorino also solid along with prosciutto. My cousin...',
  'rating': 5,
  'time_created': '2019-07-17 19:07:02',
  'user': {'id': '0Xj57cNCQ2LIfvDzytDqkg',
   'profile_url': 'https://www.yelp.com/user_details?userid=0Xj57cNCQ2LIfvDzytDqkg',
   'image_url': 'https://s3-media1.fl.yelpcdn.com/photo/pHhJKiSHv5Ahay206nNOWA/o.jpg',
   'name': 'Paul-Michael O.'},
  'business_id': '16ZnHpuaaBt92XWeJHCC5A'},
 {'id': '1yejuE9NWayMrgk80FJHTw',
  'url': 'https://www.yelp.com/biz/olio-e-pi%C3%BA-new-york-3?adjust_creative=RUNeA23i4vSpUWOOA2zPTA&hrid=1yejuE9NWayMrgk80FJHTw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_reviews&utm